In [10]:
import pyodbc
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd

class SQLConnection:
    def __init__(self,sql_path, db_server=os.getenv("DB_SERVER"), db_name=os.getenv("DB_NAME"), db_driver=os.getenv("DB_DRIVER"),params={"medico": "PSICOLOGÍA","fechaini": "20230101","fechafin": "20250504"}):
        load_dotenv()
        self.db_name = db_name
        self.db_server = db_server
        self.db_driver = db_driver
        self.params = params
        self.sql_path = sql_path
        
    def get_connection_string(self):
        db_connection_string = (f"mssql+pyodbc://@{self.db_server}/{self.db_name}"f"?driver={self.db_driver.replace(' ', '+')}&trusted_connection=yes")
        return db_connection_string
    
    def create_engine_connection(self):
        connection_string = self.get_connection_string()
        engine = create_engine(connection_string)
        return engine
    
    def generate_dataframe(self):
        with open(self.sql_path, "r", encoding="utf-8") as file: 
            query = file.read()
        
        engine = self.create_engine_connection()
        with engine.connect() as connection:
            df = pd.read_sql(text(query), connection, params=self.params)
        return df
    
    
        

In [11]:
params={"medico": "PSICOLOGÍA","fechaini": "20230101","fechafin": "20250504"}
sql_path=os.path.join("..", "sql_queries", "queries.sql")
sqlconection=SQLConnection(sql_path=sql_path, params=params)
historias_clinicas_df = sqlconection.generate_dataframe()

In [12]:
historias_clinicas_df.head()

,SEXO,EDAD,GRUPO,ESPECIALIDAD_MEDICA,SUBJETIVO,OBJETIVO,Concatenada
0,Masculino,10,T. Externalizantes,PSICOLOGÍA,None,None,
1,Femenino,38,Otros Trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...","Paciente refiere: ""Me empezaron a dar como uno..."
2,Masculino,22,T. Externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...","Paciente refiere ""Me he sentido muy mal, en el..."
3,Masculino,9,Otros Trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",None,"La madre refiere ""el viene por un acompañamien..."
4,Masculino,28,Otros Trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...","Paciente refiere ""Estas cosas que han pasado m..."
